In [1]:
import os
import pandas as pd
import geopandas as gpd
from IPython.display import display
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time
import geopy
from openrouteservice import client

In [2]:
# read and check datasets in the raw file
external = pd.DataFrame()
path = os.getcwd().replace("notebooks","") + "data/raw/"
os.listdir(path)

['newest.json',
 'feature.json',
 'income.xlsx',
 'shapefile.zip',
 '.DS_Store',
 'lowest_prices.json',
 'covid_test.csv',
 'cemetery.csv',
 'shapefiles',
 'Median_Suburb_Quarterly_2020.xls',
 'Median_Suburb_Time_2021.xls',
 'Rental_Report_Quarterly_Median_2020.xlsx',
 '.gitkeep',
 'school_2021.csv',
 'functional.xlsx',
 'Rental_Report_Quarterly_Median_2021.xlsx',
 'school_2020.csv',
 'school_2022.csv',
 'Affordable_Lettings_2020.xlsx',
 'Gaming_Expenditure.xls',
 'Population_density_gaming_expenditures.xls',
 'highest_price.json',
 'datalink.txt',
 'Rental_Report_Quarterly.xlsx',
 'Government_Organisations.xlsx',
 'GNR.csv',
 'Sport_Recreational_Facilities_list.xlsx',
 'Affordable_Lettings_2021.xlsx',
 'Median_Suburb_Quarterly_2021.xlsx',
 'australian_postcodes.csv',
 'suburb.json',
 'ambulance.csv',
 'crime.xlsx',
 'population.xlsx']

In [3]:
# read Australian_postcodes
postcode = pd.read_csv("../data/raw/australian_postcodes.csv")
postcode.columns

Index(['id', 'postcode', 'locality', 'state', 'long', 'lat', 'dc', 'type',
       'status', 'sa3', 'sa3name', 'sa4', 'sa4name', 'region', 'Lat_precise',
       'Long_precise', 'SA1_MAINCODE_2011', 'SA1_MAINCODE_2016',
       'SA2_MAINCODE_2016', 'SA2_NAME_2016', 'SA3_CODE_2016', 'SA3_NAME_2016',
       'SA4_CODE_2016', 'SA4_NAME_2016', 'RA_2011', 'RA_2016', 'MMM_2015',
       'MMM_2019', 'ced', 'altitude', 'chargezone', 'phn_code', 'phn_name',
       'lgaregion', 'electorate', 'electoraterating'],
      dtype='object')

In [4]:
# extract postcode, SA2_MAINCODE_2016, SA2_NAME_2016 from postcode to external
external['postcode'] = postcode['postcode']
external['SA2_Code'] = postcode['SA2_MAINCODE_2016']
external['SA2_Name'] = postcode['SA2_NAME_2016']
external

,postcode,SA2_Code,SA2_Name
0,200,801051049.0,Acton
1,200,801051049.0,Acton
2,800,701011002.0,Darwin City
3,800,701011002.0,Darwin City
4,801,701011002.0,Darwin City
...,...,...,...
18437,9013,305011105.0,Brisbane City
18438,9015,305011105.0,Brisbane City
18439,9464,302031038.0,Northgate - Virginia
18440,9726,309101268.0,Bundall


In [5]:
# read data, Table 1 in population
population = pd.read_excel("../data/raw/population.xlsx",sheet_name="Table 1",header=7)
#population.columns
population = population.drop(columns=['SA2 name','S/T code', 'S/T name', 'GCCSA code', 'GCCSA name', 'SA4 code','SA4 name',
                                      'SA3 code', 'SA3 name',"Unnamed: 31","Unnamed: 34"])
population.head(5)

,SA2 code,no.,no..1,no..2,no..3,no..4,no..5,no..6,no..7,no..8,...,no..15,no..16,no..17,no..18,no..19,no..20,no..21,%,km2,persons/km2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,101021007.0,2760.0,2811.0,2835.0,2844.0,2847.0,2965.0,3102.0,3181.0,3308.0,...,3950.0,4039.0,4140.0,4211.0,4273.0,4330.0,872.0,25.2,3418.4,1.3
2,101021008.0,9129.0,9199.0,9263.0,9277.0,9209.0,9212.0,9033.0,8994.0,9030.0,...,8531.0,8526.0,8507.0,8488.0,8519.0,8546.0,-546.0,-6.0,7.0,1223.9
3,101021009.0,9717.0,9513.0,9522.0,9400.0,9595.0,9682.0,9793.0,10074.0,10288.0,...,11230.0,11355.0,11447.0,11450.0,11437.0,11370.0,655.0,6.1,4.8,2387.7
4,101021010.0,3925.0,4073.0,4219.0,4218.0,4187.0,4319.0,4459.0,4595.0,4712.0,...,4970.0,5013.0,5072.0,5117.0,5077.0,5093.0,142.0,2.9,13.0,391.7


In [6]:
# change the column name
population.columns = ['SA2_Code','2001_population','2002_population','2003_population',
                      '2004_population','2005_population','2006_population', '2007_population','2008_population',
                      '2009_population', '2010_population', '2011_population', '2012_population', '2013_population',
                      '2014_population', '2015_population', '2016_population', '2017_population','2018_population',
                      '2019_population', '2020_population', '2021_population',"Population_Change_no",
                      "Population_Change_percent","Population_Area_km2","Population_density_2021_persons/km2"]

# remove the null columns
population = population.dropna()
display(population.tail(5))
population.shape

,SA2_Code,2001_population,2002_population,2003_population,2004_population,2005_population,2006_population,2007_population,2008_population,2009_population,...,2016_population,2017_population,2018_population,2019_population,2020_population,2021_population,Population_Change_no,Population_Change_percent,Population_Area_km2,Population_density_2021_persons/km2
2450,801111141.0,12.0,11.0,11.0,10.0,10.0,9.0,15.0,22.0,27.0,...,39.0,45.0,50.0,56.0,61.0,67.0,26.0,63.4,1202.8,0.1
2451,901011001.0,1442.0,1365.0,1337.0,1355.0,1380.0,1403.0,1569.0,1745.0,1944.0,...,1903.0,1877.0,1849.0,1801.0,1752.0,1716.0,-451.0,-20.8,136.1,12.6
2452,901021002.0,600.0,568.0,558.0,573.0,588.0,590.0,575.0,568.0,565.0,...,546.0,569.0,571.0,599.0,605.0,602.0,41.0,7.3,13.7,43.9
2453,901031003.0,542.0,464.0,441.0,428.0,413.0,386.0,370.0,370.0,367.0,...,402.0,392.0,376.0,360.0,338.0,310.0,-79.0,-20.3,67.2,4.6
2454,901041004.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1757.0,1827.0,1920.0,1999.0,2096.0,2220.0,2220.0,0.0,38.7,57.4


(2454, 26)

In [7]:
# merge the external and population dataset together, fill null with NaN
external = pd.merge(external, population, on='SA2_Code', how='left').fillna(np.nan)
external

,postcode,SA2_Code,SA2_Name,2001_population,2002_population,2003_population,2004_population,2005_population,2006_population,2007_population,...,2016_population,2017_population,2018_population,2019_population,2020_population,2021_population,Population_Change_no,Population_Change_percent,Population_Area_km2,Population_density_2021_persons/km2
0,200,801051049.0,Acton,1488.0,1578.0,1677.0,1777.0,1859.0,1920.0,1976.0,...,2126.0,2278.0,2450.0,2641.0,2778.0,2875.0,789.0,37.8,2.7,1050.3
1,200,801051049.0,Acton,1488.0,1578.0,1677.0,1777.0,1859.0,1920.0,1976.0,...,2126.0,2278.0,2450.0,2641.0,2778.0,2875.0,789.0,37.8,2.7,1050.3
2,800,701011002.0,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,3052.0,...,7591.0,7962.0,7856.0,7707.0,7709.0,7679.0,2626.0,52.0,3.2,2420.0
3,800,701011002.0,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,3052.0,...,7591.0,7962.0,7856.0,7707.0,7709.0,7679.0,2626.0,52.0,3.2,2420.0
4,801,701011002.0,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,3052.0,...,7591.0,7962.0,7856.0,7707.0,7709.0,7679.0,2626.0,52.0,3.2,2420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18437,9013,305011105.0,Brisbane City,2814.0,3206.0,4575.0,5448.0,6685.0,7455.0,8541.0,...,10939.0,11661.0,12371.0,13122.0,14064.0,13779.0,4423.0,47.3,2.4,5823.8
18438,9015,305011105.0,Brisbane City,2814.0,3206.0,4575.0,5448.0,6685.0,7455.0,8541.0,...,10939.0,11661.0,12371.0,13122.0,14064.0,13779.0,4423.0,47.3,2.4,5823.8
18439,9464,302031038.0,Northgate - Virginia,5631.0,5639.0,5631.0,5690.0,5785.0,5858.0,5991.0,...,6869.0,6900.0,7027.0,7172.0,7195.0,7169.0,575.0,8.7,6.1,1170.1
18440,9726,309101268.0,Bundall,4188.0,4218.0,4235.0,4250.0,4273.0,4290.0,4306.0,...,4704.0,4885.0,4957.0,4968.0,4986.0,4930.0,566.0,13.0,3.9,1263.5


In [8]:
# read data, Table 1.4 in income
income = pd.read_excel("../data/raw/income.xlsx",sheet_name="Table 1.4",header=6)
display(income.head(2))
# extract the colunms of year
income = income.drop(columns=['SA2 NAME','2014-15', '2015-16', '2016-17', '2017-18','2018-19', '2014-15.1', 
                              '2015-16.1', '2016-17.1', '2017-18.1','2018-19.1', '2014-15.2', '2015-16.2', 
                              '2016-17.2', '2017-18.2','2018-19.2','Unnamed: 27'])
income.columns = ['SA2_Code','2014-15_median_income','2015-16_median_income','2016-17_median_income',
                  '2017-18_median_income','2018-19_median_income','2014-15_mean_income','2015-16_mean_income',
                  '2016-17_mean_income','2017-18_mean_income','2018-19_mean_income']

# replace np with NaN
income = income.replace('np', np.nan)
income.head(3)

,SA2,SA2 NAME,2014-15,2015-16,2016-17,2017-18,2018-19,2014-15.1,2015-16.1,2016-17.1,...,2015-16.3,2016-17.3,2017-18.3,2018-19.3,2014-15.4,2015-16.4,2016-17.4,2017-18.4,2018-19.4,Unnamed: 27
0,Australia,NaN,13102895,13358252,13678024,14069082,14425037,42,42,42,...,47692,48360,49805,51389,61036,61975,62594,64246,65953,NaN
1,New South Wales,NaN,4091347,4191542,4344997,4466941,4569650,42,42,42,...,48085,48700,50153,51818,62798,64493,65196,67200,68816,NaN


,SA2_Code,2014-15_median_income,2015-16_median_income,2016-17_median_income,2017-18_median_income,2018-19_median_income,2014-15_mean_income,2015-16_mean_income,2016-17_mean_income,2017-18_mean_income,2018-19_mean_income
0,Australia,46854.0,47692.0,48360,49805.0,51389.0,61036.0,61975.0,62594.0,64246.0,65953.0
1,New South Wales,46879.0,48085.0,48700,50153.0,51818.0,62798.0,64493.0,65196.0,67200.0,68816.0
2,101021007,38093.0,39716.0,41288,42003.0,41593.0,47741.0,51074.0,51090.0,51594.0,51149.0


In [9]:
# merge the exrternal with income data
external = pd.merge(external, income, on='SA2_Code', how='left').fillna(np.nan)
external

,postcode,SA2_Code,SA2_Name,2001_population,2002_population,2003_population,2004_population,2005_population,2006_population,2007_population,...,2014-15_median_income,2015-16_median_income,2016-17_median_income,2017-18_median_income,2018-19_median_income,2014-15_mean_income,2015-16_mean_income,2016-17_mean_income,2017-18_mean_income,2018-19_mean_income
0,200,801051049.0,Acton,1488.0,1578.0,1677.0,1777.0,1859.0,1920.0,1976.0,...,10384.0,10332.0,8682,9306.0,10433.0,21494.0,21438.0,17656.0,16835.0,19479.0
1,200,801051049.0,Acton,1488.0,1578.0,1677.0,1777.0,1859.0,1920.0,1976.0,...,10384.0,10332.0,8682,9306.0,10433.0,21494.0,21438.0,17656.0,16835.0,19479.0
2,800,701011002.0,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,3052.0,...,51473.0,56408.0,56035,60937.0,57789.0,70661.0,78636.0,81685.0,87791.0,74682.0
3,800,701011002.0,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,3052.0,...,51473.0,56408.0,56035,60937.0,57789.0,70661.0,78636.0,81685.0,87791.0,74682.0
4,801,701011002.0,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,3052.0,...,51473.0,56408.0,56035,60937.0,57789.0,70661.0,78636.0,81685.0,87791.0,74682.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18437,9013,305011105.0,Brisbane City,2814.0,3206.0,4575.0,5448.0,6685.0,7455.0,8541.0,...,35116.0,35942.0,33956,37501.0,39592.0,69218.0,65963.0,65904.0,69649.0,71479.0
18438,9015,305011105.0,Brisbane City,2814.0,3206.0,4575.0,5448.0,6685.0,7455.0,8541.0,...,35116.0,35942.0,33956,37501.0,39592.0,69218.0,65963.0,65904.0,69649.0,71479.0
18439,9464,302031038.0,Northgate - Virginia,5631.0,5639.0,5631.0,5690.0,5785.0,5858.0,5991.0,...,54533.0,55445.0,55722,57498.0,59389.0,65369.0,66354.0,66705.0,68703.0,71806.0
18440,9726,309101268.0,Bundall,4188.0,4218.0,4235.0,4250.0,4273.0,4290.0,4306.0,...,43959.0,46370.0,44066,44184.0,48078.0,67719.0,71221.0,68178.0,71310.0,75576.0


In [10]:
# helper function for count the total area by SA2 Area
def sum_area(df):
    df["total"] = 0
    col = df.columns
    df["total"] = df[col[1]]+df[col[2]]+df[col[3]]+df[col[4]]+df[col[5]]+df[col[6]]+df[col[7]]
    return df

In [11]:
# read Land and Housing Supply Indicators data and extract the data we needed
residential_area = pd.read_excel("../data/raw/functional.xlsx",sheet_name="Table 1",header=5)
residential_area.columns = ['SA2_Code', 'SA2 name', "residential_area<200sqm","residential_area_200-400sqm",
                           "residential_area_400-600sqm","residential_area_600-800sqm","residential_area_800-1000sqm",
                           "residential_area_1000-10000sqm","residential_area>10000sqm"]
residential_area = residential_area.drop(columns=['SA2 name'])
sum_area(residential_area)

primary_production_area = pd.read_excel("../data/raw/functional.xlsx",sheet_name="Table 3",header=5)
primary_production_area.columns = ['SA2_Code', 'SA2 name', "primary_production<200sqm","primary_production_200-400sqm",
                           "primary_production_400-600sqm","primary_production_600-800sqm",
                           "primary_production_800-1000sqm","primary_production_1000-10000sqm",
                           "primary_production>10000sqm"]
primary_production_area = primary_production_area.drop(columns=['SA2 name'])
sum_area(primary_production_area)

community_infrastructure_area = pd.read_excel("../data/raw/functional.xlsx",sheet_name="Table 3",header=5)
community_infrastructure_area.columns = ['SA2_Code', 'SA2 name', "community_infrastructure<200sqm",
                                         "community_infrastructure_200-400sqm","community_infrastructure_400-600sqm",
                                         "community_infrastructure_600-800sqm","community_infrastructure_800-1000sqm",
                                         "community_infrastructure_1000-10000sqm","community_infrastructure>10000sqm"]
community_infrastructure_area = community_infrastructure_area.drop(columns=['SA2 name'])
sum_area(community_infrastructure_area)

mix_use_area = pd.read_excel("../data/raw/functional.xlsx",sheet_name="Table 5",header=5)
mix_use_area.columns = ['SA2_Code', 'SA2 name','mix_area<200sqm', 'mix_area_200-400sqm', 'mix_area_400-600sqm',
                        'mix_area_600-800sqm', 'mix_area_800-1000sqm', 'mix_area_1000-10000sqm', 'mix_area>1000sqm']
mix_use_area = mix_use_area.drop(columns=['SA2 name'])
sum_area(mix_use_area)

industrial_area = pd.read_excel("../data/raw/functional.xlsx",sheet_name="Table 6",header=5)
industrial_area.columns = ['SA2_Code', 'SA2 name','industrial_area<200sqm', 'industrial_area_200-400sqm',
                           'industrial_area_400-600sqm', 'industrial_area_600-800sqm', 'industrial_area_800-1000sqm',
                           'industrial_area_1000-10000sqm', 'industrial_area>1000sqm']
industrial_area = industrial_area.drop(columns=['SA2 name'])
sum_area(industrial_area)

scenic_spot_area = pd.read_excel("../data/raw/functional.xlsx",sheet_name="Table 7",header=5)
scenic_spot_area.columns = ['SA2_Code', 'SA2 name','scenic_spot<200sqm', 'scenic_spot_200-400sqm',
                             'scenic_spot_400-600sqm', 'scenic_spot_600-800sqm', 'scenic_spot_800-1000sqm',
                             'scenic_spot_1000-10000sqm', 'scenic_spot>1000sqm']
scenic_spot_area = scenic_spot_area.drop(columns=['SA2 name'])
sum_area(scenic_spot_area)

commerical_area = pd.read_excel("../data/raw/functional.xlsx",sheet_name="Table 8",header=5)
commerical_area.columns = ['SA2_Code', 'SA2 name','commerical_area<200sqm', 'commerical_area_200-400sqm',
                           'commerical_area_400-600sqm', 'commerical_area_600-800sqm', 'commerical_area_800-1000sqm',
                           'commerical_area_1000-10000sqm', 'commerical_area>1000sqm']
commerical_area = commerical_area.drop(columns=['SA2 name'])
sum_area(commerical_area)

utilities_area = pd.read_excel("../data/raw/functional.xlsx",sheet_name="Table 10",header=5)
utilities_area.columns = ['SA2_Code', 'SA2 name','utilities_area<200sqm', 'utilities_area_200-400sqm',
                          'utilities_area_400-600sqm', 'utilities_area_600-800sqm', 'utilities_area_800-1000sqm',
                          'utilities_area_1000-10000sqm', 'utilities_area>1000sqm']
utilities_area = utilities_area.drop(columns=['SA2 name'])
sum_area(utilities_area)

,SA2_Code,utilities_area<200sqm,utilities_area_200-400sqm,utilities_area_400-600sqm,utilities_area_600-800sqm,utilities_area_800-1000sqm,utilities_area_1000-10000sqm,utilities_area>1000sqm,total
0,4001,21,23,14,13,11,45,114,241
1,403041071,0,0,0,0,0,0,0,0
2,401011001,0,0,0,0,0,0,0,0
3,404031104,0,0,0,0,0,0,0,0
4,401021004,0,0,0,0,0,3,2,5
...,...,...,...,...,...,...,...,...,...
1518,107031143,0,0,4,1,0,4,1,10
1519,107011547,0,0,1,0,0,0,2,3
1520,107041548,4,0,0,0,4,3,4,15
1521,107041549,1,2,0,1,0,3,4,11


In [12]:
#sum the functional area by SA2
functional = pd.DataFrame()
functional["SA2_Code"] = residential_area["SA2_Code"]
functional["residential"] = residential_area["total"]
functional["primary_production"] = primary_production_area["total"]
functional["community_infrastructure"] = community_infrastructure_area["total"]
functional["mix_use"] = mix_use_area["total"]
functional["industrial"] = industrial_area["total"]
functional["scenic_spot"] = scenic_spot_area["total"]
functional["commerical"] = commerical_area["total"]
functional["utilities"] = utilities_area["total"]

In [13]:
# merge the exrternal with functional area data
external = pd.merge(external, functional, on='SA2_Code', how='left').fillna(np.nan)
external

,postcode,SA2_Code,SA2_Name,2001_population,2002_population,2003_population,2004_population,2005_population,2006_population,2007_population,...,2017-18_mean_income,2018-19_mean_income,residential,primary_production,community_infrastructure,mix_use,industrial,scenic_spot,commerical,utilities
0,200,801051049.0,Acton,1488.0,1578.0,1677.0,1777.0,1859.0,1920.0,1976.0,...,16835.0,19479.0,0,0,0,0,0,0,0,0
1,200,801051049.0,Acton,1488.0,1578.0,1677.0,1777.0,1859.0,1920.0,1976.0,...,16835.0,19479.0,0,0,0,0,0,0,0,0
2,800,701011002.0,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,3052.0,...,87791.0,74682.0,14,0,0,0,3,0,479,8
3,800,701011002.0,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,3052.0,...,87791.0,74682.0,14,0,0,0,3,0,479,8
4,801,701011002.0,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,3052.0,...,87791.0,74682.0,14,0,0,0,3,0,479,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18437,9013,305011105.0,Brisbane City,2814.0,3206.0,4575.0,5448.0,6685.0,7455.0,8541.0,...,69649.0,71479.0,418,0,0,659,0,0,0,1
18438,9015,305011105.0,Brisbane City,2814.0,3206.0,4575.0,5448.0,6685.0,7455.0,8541.0,...,69649.0,71479.0,418,0,0,659,0,0,0,1
18439,9464,302031038.0,Northgate - Virginia,5631.0,5639.0,5631.0,5690.0,5785.0,5858.0,5991.0,...,68703.0,71806.0,2493,1,1,116,515,10,32,11
18440,9726,309101268.0,Bundall,4188.0,4218.0,4235.0,4250.0,4273.0,4290.0,4306.0,...,71310.0,75576.0,1503,0,0,140,0,0,0,0


In [14]:
# calculate the number of cemetery by postcode
cemetery = pd.read_csv("../data/raw/cemetery.csv")
cemetery_count=pd.DataFrame.from_dict(dict(cemetery["Postcode"].value_counts()),orient='index').reset_index()
cemetery_count.columns=["postcode","cemetery_number"]

In [15]:
# merge the exrternal with cemetery data
external = pd.merge(external, cemetery_count, on='postcode', how='left').fillna(np.nan)
external

,postcode,SA2_Code,SA2_Name,2001_population,2002_population,2003_population,2004_population,2005_population,2006_population,2007_population,...,2018-19_mean_income,residential,primary_production,community_infrastructure,mix_use,industrial,scenic_spot,commerical,utilities,cemetery_number
0,200,801051049.0,Acton,1488.0,1578.0,1677.0,1777.0,1859.0,1920.0,1976.0,...,19479.0,0,0,0,0,0,0,0,0,NaN
1,200,801051049.0,Acton,1488.0,1578.0,1677.0,1777.0,1859.0,1920.0,1976.0,...,19479.0,0,0,0,0,0,0,0,0,NaN
2,800,701011002.0,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,3052.0,...,74682.0,14,0,0,0,3,0,479,8,NaN
3,800,701011002.0,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,3052.0,...,74682.0,14,0,0,0,3,0,479,8,NaN
4,801,701011002.0,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,3052.0,...,74682.0,14,0,0,0,3,0,479,8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18437,9013,305011105.0,Brisbane City,2814.0,3206.0,4575.0,5448.0,6685.0,7455.0,8541.0,...,71479.0,418,0,0,659,0,0,0,1,NaN
18438,9015,305011105.0,Brisbane City,2814.0,3206.0,4575.0,5448.0,6685.0,7455.0,8541.0,...,71479.0,418,0,0,659,0,0,0,1,NaN
18439,9464,302031038.0,Northgate - Virginia,5631.0,5639.0,5631.0,5690.0,5785.0,5858.0,5991.0,...,71806.0,2493,1,1,116,515,10,32,11,NaN
18440,9726,309101268.0,Bundall,4188.0,4218.0,4235.0,4250.0,4273.0,4290.0,4306.0,...,75576.0,1503,0,0,140,0,0,0,0,NaN


In [16]:
# calculate the number of Sport&Recreational Facilities by postcode
facilities= pd.read_excel("../data/raw/Sport_Recreational_Facilities_list.xlsx")
facilities = facilities[['Facility Name','Pcode']]
facilities=facilities.drop_duplicates()
facilities_count=pd.DataFrame.from_dict(dict(facilities['Pcode'].value_counts()),orient='index').reset_index()
facilities_count.columns=["postcode","Sport&Recreational_facilities_number"]

In [17]:
# merge the exrternal with Sport&Recreational Facilitiesdata
external = pd.merge(external, facilities_count, on='postcode', how='left').fillna(np.nan)
external

,postcode,SA2_Code,SA2_Name,2001_population,2002_population,2003_population,2004_population,2005_population,2006_population,2007_population,...,residential,primary_production,community_infrastructure,mix_use,industrial,scenic_spot,commerical,utilities,cemetery_number,Sport&Recreational_facilities_number
0,200,801051049.0,Acton,1488.0,1578.0,1677.0,1777.0,1859.0,1920.0,1976.0,...,0,0,0,0,0,0,0,0,NaN,NaN
1,200,801051049.0,Acton,1488.0,1578.0,1677.0,1777.0,1859.0,1920.0,1976.0,...,0,0,0,0,0,0,0,0,NaN,NaN
2,800,701011002.0,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,3052.0,...,14,0,0,0,3,0,479,8,NaN,NaN
3,800,701011002.0,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,3052.0,...,14,0,0,0,3,0,479,8,NaN,NaN
4,801,701011002.0,Darwin City,2140.0,2248.0,2320.0,2429.0,2661.0,2798.0,3052.0,...,14,0,0,0,3,0,479,8,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18437,9013,305011105.0,Brisbane City,2814.0,3206.0,4575.0,5448.0,6685.0,7455.0,8541.0,...,418,0,0,659,0,0,0,1,NaN,NaN
18438,9015,305011105.0,Brisbane City,2814.0,3206.0,4575.0,5448.0,6685.0,7455.0,8541.0,...,418,0,0,659,0,0,0,1,NaN,NaN
18439,9464,302031038.0,Northgate - Virginia,5631.0,5639.0,5631.0,5690.0,5785.0,5858.0,5991.0,...,2493,1,1,116,515,10,32,11,NaN,NaN
18440,9726,309101268.0,Bundall,4188.0,4218.0,4235.0,4250.0,4273.0,4290.0,4306.0,...,1503,0,0,140,0,0,0,0,NaN,NaN


In [18]:
#save external data
path = os.getcwd().replace("notebooks","") + "data/curated/"
external.to_csv(path+'external.csv',index=False)

#### Preprocess for GNR external dataset

In [19]:
# read GNR shapefile
GNR = gpd.read_file('../data/raw/shapefiles/gda2020_vicgrid/esrishape/whole_of_dataset/victoria/VMFOI/GNR.shp')
display(GNR.head(5))
GNR = GNR.drop(columns=['UFI', 'NAME_ID', 'VICMAP_ID', 'FEATUREC', 'NAMESTATC', 'NAMESTAT','REG_DATE','CRDATE_UFI'])
display(GNR.head(5))
path = os.getcwd().replace("notebooks","") + "data/raw/"
GNR.to_csv(path+"GNR.csv",index=False)

,UFI,PLACE_ID,NAME_ID,VICMAP_ID,PLACE_NAME,FEATUREC,FEATURE,NAMESTATC,NAMESTAT,LONGITUDE,LATITUDE,REG_DATE,CRDATE_UFI,geometry
0,416079,338,1064,655117.0,BALLARAT FIRE STATION,FIST,FIRE STATION,REG,REGISTERED,143.8686,-37.563,2018-08-13,2022-02-11,POINT (2400054.382 2436830.207)
1,416080,1256,4074,621722.0,BUNINYONG CREEK,WACO,WATERCOURSE,H,HISTORICAL,143.8582,-37.659,2002-05-24,2022-02-11,POINT (2399261.401 2426265.924)
2,416081,1257,4075,1167467.0,BUNINYONG FIRE STATION,FIST,FIRE STATION,REG,REGISTERED,143.8851,-37.650,2018-08-13,2022-02-11,POINT (2401628.205 2427218.508)
3,416082,3223,10605,727803.0,GRAND STAIRWAY,CLIF,CLIFF,REG,REGISTERED,142.5034,-37.191,1966-05-02,2022-02-11,POINT (2278388.543 2475893.428)
4,416083,3728,12332,655565.0,INVERMAY FIRE STATION,FIST,FIRE STATION,REG,REGISTERED,143.8724,-37.521,2018-08-13,2022-02-11,POINT (2400342.015 2441487.169)


,PLACE_ID,PLACE_NAME,FEATURE,LONGITUDE,LATITUDE,geometry
0,338,BALLARAT FIRE STATION,FIRE STATION,143.8686,-37.563,POINT (2400054.382 2436830.207)
1,1256,BUNINYONG CREEK,WATERCOURSE,143.8582,-37.659,POINT (2399261.401 2426265.924)
2,1257,BUNINYONG FIRE STATION,FIRE STATION,143.8851,-37.650,POINT (2401628.205 2427218.508)
3,3223,GRAND STAIRWAY,CLIFF,142.5034,-37.191,POINT (2278388.543 2475893.428)
4,3728,INVERMAY FIRE STATION,FIRE STATION,143.8724,-37.521,POINT (2400342.015 2441487.169)


In [20]:
# read GNR csv
gnr = pd.read_csv('../data/raw/GNR.csv')

# extract each useful feature (59 needed features)
fire_st = gnr[gnr['FEATURE']=='FIRE STATION']
sec_sch = gnr[gnr['FEATURE']=='SECONDARY SCHOOL']
PRIMARY_SCHOOL = gnr[gnr['FEATURE']=='PRIMARY SCHOOL']
SHOPPING_CENTRE = gnr[gnr['FEATURE']=='SHOPPING CENTRE']
PARK = gnr[gnr['FEATURE']=='PARK']
KINDERGARTEN = gnr[gnr['FEATURE']=='KINDERGARTEN']
POST_OFFICE = gnr[gnr['FEATURE']=='POST OFFICE']
SPECIAL_SCHOOL = gnr[gnr['FEATURE']=='SPECIAL SCHOOL']
PRIMARY_AND_SECONDARY_SCHOOL = gnr[gnr['FEATURE']=='PRIMARY AND SECONDARY SCHOOL']
CHILD_CARE= gnr[gnr['FEATURE']=='CHILD CARE']
RAILWAY= gnr[gnr['FEATURE']=='RAILWAY']
LOCAL_GOVERNMENT_AREA = gnr[gnr['FEATURE']=='LOCAL GOVERNMENT AREA']
SPORTS_COMPLEX = gnr[gnr['FEATURE']=='SPORTS COMPLEX']
EDUCATION_COMPLEX = gnr[gnr['FEATURE']=='EDUCATION COMPLEX']
CEMETERY = gnr[gnr['FEATURE']=='CEMETERY']
LIBRARY = gnr[gnr['FEATURE']=='LIBRARY']
UNIVERSITY = gnr[gnr['FEATURE']=='UNIVERSITY']
GENERAL_HOSPITAL = gnr[gnr['FEATURE']=='GENERAL HOSPITAL']
POLICE_STATION = gnr[gnr['FEATURE']=='POLICE STATION']
FURTHER_EDUCATION = gnr[gnr['FEATURE']=='FURTHER EDUCATION']
PLAYGROUND = gnr[gnr['FEATURE']=='PLAYGROUND']
PLANTATION = gnr[gnr['FEATURE']=='PLANTATION']
WINERY = gnr[gnr['FEATURE']=='WINERY']
BAR = gnr[gnr['FEATURE']=='BAR']
FARM = gnr[gnr['FEATURE']=='FARM']
VINEYARD= gnr[gnr['FEATURE']=='VINEYARD']
TENNIS_COURT= gnr[gnr['FEATURE']=='TENNIS COURT']
BOWLING_GREEN= gnr[gnr['FEATURE']=='BOWLING GREEN']
BAY= gnr[gnr['FEATURE']=='BAY']
PIER= gnr[gnr['FEATURE']=='PIER']
SWIMMING_POOL= gnr[gnr['FEATURE']=='SWIMMING POOL']
ART_GALLERY= gnr[gnr['FEATURE']=='ART GALLERY']
HARBOUR= gnr[gnr['FEATURE']=='HARBOUR']
WHARF= gnr[gnr['FEATURE']=='WHARF']
COMMUNITY_HEALTH_CENTRE = gnr[gnr['FEATURE']=='COMMUNITY HEALTH CENTRE']
MATERNAL_AND_CHILD_HEALTH_CENTRE= gnr[gnr['FEATURE']=='MATERNAL AND CHILD HEALTH CENTRE']
MARKET= gnr[gnr['FEATURE']=='MARKET']
PRISON= gnr[gnr['FEATURE']=='PRISON']
BOTANIC_GARDENS= gnr[gnr['FEATURE']=='BOTANIC GARDENS']
SPECIALISED_HOSPITAL= gnr[gnr['FEATURE']=='SPECIALISED HOSPITAL']
BANK= gnr[gnr['FEATURE']=='BANK']
TERTIARY_INSTITUTION= gnr[gnr['FEATURE']=='TERTIARY INSTITUTION']
AGED_CARE= gnr[gnr['FEATURE']=='AGED CARE']
SURFING_SPOT= gnr[gnr['FEATURE']=='SURFING SPOT']
AMBULANCE_STATION= gnr[gnr['FEATURE']=='AMBULANCE STATION']
COAST= gnr[gnr['FEATURE']=='COAST']
ARBORETUM= gnr[gnr['FEATURE']=='ARBORETUM']
VELODROME= gnr[gnr['FEATURE']=='VELODROME']
GOLF_COURSE= gnr[gnr['FEATURE']=='GOLF COURSE']
EQUESTRIAN_CENTRE  = gnr[gnr['FEATURE']=='EQUESTRIAN CENTRE']        
LIGHT_RAIL_STATION= gnr[gnr['FEATURE']=='LIGHT RAIL STATION']
HELIPORT= gnr[gnr['FEATURE']=='HELIPORT']
AIRPORT= gnr[gnr['FEATURE']=='AIRPORT']
FIRING_RANGE= gnr[gnr['FEATURE']=='FIRING RANGE']
IRON_ORE_PROCESSOR= gnr[gnr['FEATURE']=='IRON ORE PROCESSOR']
MILL_TIMBER_OPERATIONS= gnr[gnr['FEATURE']=='MILL/TIMBER OPERATIONS']
FERRY_STATION= gnr[gnr['FEATURE']=='FERRY STATION']
BEACH= gnr[gnr['FEATURE']=='BEACH']
MARINA = gnr[gnr['FEATURE']=='MARINA']

In [21]:
# create a new list to put all extrated features
d_lst=[fire_st,sec_sch, PRIMARY_AND_SECONDARY_SCHOOL, PRIMARY_SCHOOL, SHOPPING_CENTRE, PARK, KINDERGARTEN, POST_OFFICE, SPECIAL_SCHOOL, CHILD_CARE, RAILWAY,
BAR, LOCAL_GOVERNMENT_AREA, SPORTS_COMPLEX, EDUCATION_COMPLEX, CEMETERY, LIBRARY, UNIVERSITY, GENERAL_HOSPITAL, POLICE_STATION, FURTHER_EDUCATION, PLAYGROUND, 
PLANTATION, WINERY, BAR, FARM, VINEYARD, TENNIS_COURT, BOWLING_GREEN, BAY, PIER, SWIMMING_POOL, ART_GALLERY, HARBOUR, WHARF, COMMUNITY_HEALTH_CENTRE,
MATERNAL_AND_CHILD_HEALTH_CENTRE, MARKET, PRISON, BOTANIC_GARDENS, SPECIALISED_HOSPITAL, BANK, TERTIARY_INSTITUTION, AGED_CARE, SURFING_SPOT,
AMBULANCE_STATION, COAST, ARBORETUM, VELODROME, GOLF_COURSE, EQUESTRIAN_CENTRE, LIGHT_RAIL_STATION, HELIPORT, AIRPORT, FIRING_RANGE, IRON_ORE_PROCESSOR,
MILL_TIMBER_OPERATIONS, FERRY_STATION, BEACH, MARINA]

# combine all the features into one dataframe and remove the useless colunms
clean_GNR = pd.concat(d_lst)

# replace marina(4) and wharf(11) into harbour(18 +4+11)
clean_GNR = clean_GNR.replace('MARINA', 'HARBOUR')
clean_GNR = clean_GNR.replace('WHARF', 'HARBOUR')

clean_GNR

,PLACE_ID,PLACE_NAME,FEATURE,LONGITUDE,LATITUDE,geometry
0,338,BALLARAT FIRE STATION,FIRE STATION,143.8686,-37.563,POINT (2400054.381512979 2436830.206961736)
2,1257,BUNINYONG FIRE STATION,FIRE STATION,143.8851,-37.650,POINT (2401628.205143962 2427218.507639693)
4,3728,INVERMAY FIRE STATION,FIRE STATION,143.8724,-37.521,POINT (2400342.014512663 2441487.168515854)
7,5179,MINERS REST FIRE STATION,FIRE STATION,143.7989,-37.479,POINT (2393779.920422106 2446072.622993102)
17,8520,WENDOUREE FIRE STATION,FIRE STATION,143.8197,-37.535,POINT (2395699.25464796 2439932.614528429)
...,...,...,...,...,...,...
47751,30554,Sandy Waterhole Beach,BEACH,145.4372,-38.540,POINT (2538127.531405735 2328989.950449847)
11672,6022,No 2 Quay,HARBOUR,141.6193,-38.353,POINT (2204521.130760126 2344540.841320216)
31009,13511,CORIO QUAY,HARBOUR,144.3644,-38.106,POINT (2444259.084106595 2377028.919269247)
39662,126171,Bay City Marina,HARBOUR,144.3677,-38.143,POINT (2444573.566641665 2372862.745070743)


In [22]:
def find_suburb(address):
    suburb = np.nan
    try:
        if "suburb" in address.keys():
            suburb = address["suburb"]     
    except Exception as e:
        print(e)
        print(location)
    return suburb

def find_postcode(address):
    postcode = np.nan
    try:
        if "postcode" in address.keys():
            postcode = address["postcode"]
    except:
        pass
    return postcode
def find_municipality(address):
    municipality = np.nan
    try:
        if surburb == np.nan and ("municipality" in address.keys() or "city_district" in address.keys() 
                                  or "city" in address.keys()):
            if "city" in address.keys():
                municipality = location.raw["address"]["city"]
            elif "city_district" in address.keys():
                municipality = location.raw["address"]["city_district"]
            elif "municipalit" in address.keys():
                municipality = location.raw["address"]["municipality"]
            else:
                municipality = np.nan
    except:
        pass
    return municipality

def reverse_coord(coordinates):
    geopy.geocoders.options.default_user_agent = "my"
    geolocator = Nominatim(user_agent="my")
    reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)
    location=reverse(coordinates,language='en',exactly_one=True)
    address = location.raw["address"]
    suburb = find_suburb(address)
    postcode = find_postcode(address)
    municipality = find_municipality(address)
    
    result = {"suburb":suburb, "postcode":postcode, "municipality":municipality}
    
    return result

def generate_suburb_postcode(df):
    df["coordinates"] = df["LATITUDE"].astype(str) + ", " + df["LONGITUDE"].astype(str)
    df["Reverse_Coord"]= df["coordinates"].apply(reverse_coord)
    return df

In [23]:

clean_GNR = generate_suburb_postcode(clean_GNR)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('-37.893, 145.9991',), **{'language': 'en', 'exactly_one': True}).
Traceback (most recent call last):
  File "/Users/lucinda/opt/anaconda3/envs/ads_env/lib/python3.8/site-packages/urllib3/connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/lucinda/opt/anaconda3/envs/ads_env/lib/python3.8/site-packages/urllib3/connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/lucinda/opt/anaconda3/envs/ads_env/lib/python3.8/http/client.py", line 1344, in getresponse
    response.begin()
  File "/Users/lucinda/opt/anaconda3/envs/ads_env/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/lucinda/opt/anaconda3/envs/ads_env/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/

KeyboardInterrupt: 

In [ ]:
#save GNR Data
path = os.getcwd().replace("notebooks","") + "data/curated/"
external.to_csv(path+'GNR.csv',index=False)

In [28]:
coordinates = "-37.8, 144.96"
geopy.geocoders.options.default_user_agent = "my"
geolocator = Nominatim(user_agent="my")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)
location=reverse(coordinates,language='en',exactly_one=True)
location.raw["address"]

{'railway': 'Parkville',
 'road': 'Grattan Street',
 'commercial': 'Melbourne Innovation District',
 'suburb': 'Parkville',
 'city': 'Melbourne',
 'municipality': 'City of Melbourne',
 'state': 'Victoria',
 'ISO3166-2-lvl4': 'AU-VIC',
 'postcode': '3050',
 'country': 'Australia',
 'country_code': 'au'}

In [34]:
from openrouteservice import client
import openrouteservice
coords = ((8.34234,48.23424),(8.34423,48.26424))
client = openrouteservice.Client(key='5b3ce3597851110001cf6248d864908ae526479e86e6f4dd70971a37') 
#openrouteservice.geocode.pelias_reverse(client,(-37.8, 144.96))
client.directions(coords)

{'routes': [{'summary': {'distance': 5479.0, 'duration': 803.8},
   'segments': [{'distance': 5479.0,
     'duration': 803.8,
     'steps': [{'distance': 247.3,
       'duration': 59.4,
       'type': 11,
       'instruction': 'Head south on Benatweg',
       'name': 'Benatweg',
       'way_points': [0, 9]},
      {'distance': 51.8,
       'duration': 12.4,
       'type': 6,
       'instruction': 'Continue straight onto Benatweg',
       'name': 'Benatweg',
       'way_points': [9, 12]},
      {'distance': 1684.6,
       'duration': 134.8,
       'type': 1,
       'instruction': 'Turn right onto Hölzle, K 5528',
       'name': 'Hölzle, K 5528',
       'way_points': [12, 72]},
      {'distance': 263.8,
       'duration': 31.7,
       'type': 1,
       'instruction': 'Turn right',
       'name': '-',
       'way_points': [72, 81]},
      {'distance': 302.0,
       'duration': 36.2,
       'type': 0,
       'instruction': 'Turn left',
       'name': '-',
       'way_points': [81, 96]},
  

In [44]:
import osmnx as ox
import networkx as nx
import rtree
import pygeos
ox.config(log_console=True, use_cache=True)
# define the start and end locations in latlng
start_latlng = (37.78497,-122.43327)
end_latlng = (37.78071,-122.41445)
# location where you want to find your route
place = 'San Francisco, California, United States'
# find shortest route based on the mode of travel
mode = 'walk' # 'drive', 'bike', 'walk'
# find shortest path based on distance or time
optimizer = 'time' # 'length','time'
# create graph from OSM within the boundaries of some
# geocodable place(s)
graph = ox.graph_from_place(place, network_type = mode)
# find the nearest node to the start location
orig_node = ox.get_nearest_node(graph, start_latlng)
# find the nearest node to the end location
dest_node = ox.get_nearest_node(graph, end_latlng)
# find the shortest path
shortest_route = nx.shortest_path(graph,orig_node,dest_node,weight=optimizer)
shortest_route_map = ox.plot_route_folium(graph, shortest_route)
shortest_route_map

/Users/lucinda/opt/anaconda3/envs/ads_env/lib/python3.8/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


ImportError: Spatial indexes require either `rtree` or `pygeos`. See installation instructions at https://geopandas.org/install.html